In [ ]:
"""
Analysis of Clytia planula single cell RNAseq data using ScanPy
Marc Meynadier
"""

In [ ]:
!date

In [ ]:
import numpy as np
import anndata
import pandas as pd
import scanpy as sc
import scipy.sparse
import scanpy.external as sce
import os

import warnings
warnings.filterwarnings('ignore')

from sklearn.neighbors import (KNeighborsClassifier,NeighborhoodComponentsAnalysis)
from sklearn.pipeline import Pipeline
from sklearn.manifold import TSNE

import matplotlib.pyplot as plt
from matplotlib import rcParams
#import plotly.tools as tls

import seaborn as sns
sns.set(style="whitegrid")

import collections
import scipy.sparse as sp_sparse
import h5py

In [ ]:
sc.logging.print_header()
sc.settings.set_figure_params(dpi=150, facecolor='white')

In [ ]:
# Loading planula data

plaPath = "../../../../species/Clytia/analysis/STARmapping/chari2021/Planula/"
adata = sc.read_10x_mtx(plaPath) ; adata.var_names_make_unique()

In [ ]:
# MetaData
adata.obs['type']="planula"
adata.obs['sample']="LBDV"

In [ ]:
# Mitochondrial genes
adata.var['mt'] = adata.var_names.str.startswith('XLOC_MITO')
adata.var

In [ ]:
# Mitochondrial quality check
sc.pp.calculate_qc_metrics(adata, qc_vars=['mt'], percent_top=None, log1p=True, inplace=True)
mito_genes = adata.var_names.str.startswith('XLOC_MITO')
# For each cell compute fraction of counts in mito genes vs. all genes
adata.obs['percent_mt2'] = np.sum(adata[:, mito_genes].X, axis=1).A1 / np.sum(adata.X, axis=1).A1
# Add the total counts per cell as observations-annotation 
adata.obs['n_counts'] = adata.X.sum(axis=1).A1
adata

In [ ]:
sc.pl.violin(adata, ['n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts'],
             jitter=0.4, groupby = 'sample', rotation= 45, ncols=2)
sc.pl.violin(adata, ['pct_counts_mt', 'percent_mt2','log1p_total_counts_mt'],
             jitter=0.4, groupby = 'sample', rotation= 45, ncols=2)

In [ ]:
# Plot quality check measures
sc.pl.scatter(adata, x='total_counts', y='pct_counts_mt', color="sample")
sc.pl.scatter(adata, x='total_counts', y='n_genes_by_counts', color="sample")
sc.pl.scatter(adata, x='log1p_total_counts', y='log1p_n_genes_by_counts', color="sample")

In [ ]:
# Filtering by genes counts and mitochrondial genes counts
adata = adata[adata.obs.n_genes_by_counts < 9000, :]
adata = adata[adata.obs.pct_counts_mt < 1, :]
adata

In [ ]:
# Filtering by number of counts and number of cells
sc.pp.filter_cells(adata, min_counts=500)
sc.pp.filter_genes(adata, min_cells=2)
print(adata.n_obs,adata.n_vars)

In [ ]:
# Plotting percentage of counts per gene 
sc.pl.highest_expr_genes(adata, n_top=50)

In [ ]:
# Exporting raw adata
rPath = os.getcwd()
outputPath = rPath+'/../../../../species/Clytia/analysis/ScanPy/'
adata.write_h5ad(outputPath+'Clytia_planula_raw.h5ad')

In [ ]:
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)
sc.pp.highly_variable_genes(adata, min_mean=0.0125, max_mean=3, min_disp=0.5)

adata.raw = adata

adata = adata[:, adata.var.highly_variable]
sc.pp.regress_out(adata, ['total_counts', 'pct_counts_mt'])
sc.pp.scale(adata, max_value=10)
sc.tl.pca(adata, svd_solver='arpack')

In [ ]:
# Correcting batch effet
sce.pp.harmony_integrate(adata,'sample',max_iter_harmony=20)

In [ ]:
adata

In [ ]:
# Computing neighbors graph
sc.pp.neighbors(adata, n_neighbors=50, n_pcs=40, knn=True, use_rep='X_pca_harmony')

# Clustering the graph using Leiden algorithm
sc.tl.leiden(adata,resolution=1.5) 

# Computing UMAP
sc.tl.umap(adata)

In [ ]:
leiden = (
    adata.obs['leiden']
    .map(lambda x: {"0": "0 - Endomesoderm", "1":"1 - Secretory","2": "2 - Endomesoderm", "3":"3 - Ectoderm","4":"4 - Ectoderm","5":"5 - I-cells","6":"6 - I-cells","7":"7 - NPC","8":"8 - Ectoderm","9": "Endomesoderm", "10":"Early cnidocyte","11":"Mucous gland","12":"Cnidocytes 2","13":"Mature cnidocytes","14":"Cnidocytes 1","15":"15 - Neurons","16":"16 - Neurons","17": "17 - Neurons", "18":"18 - Neurons","19":"19 - Neurons","20":"20 - PEC-B","21":"21 - PEC-A"}.get(x, x))
    .astype("category")
)

In [ ]:
# Specific to clytia planula : Get rid of contaminated clusters 0 and 2 => Reload raw data, give annotation, get rid of clusters and redo pre-processing

adata = sc.read_h5ad(outputPath+'Clytia_planula_raw.h5ad')
adata.obs['leiden'] = leiden
adata = adata[~adata.obs['leiden'].isin(['0 - Endomesoderm','2 - Endomesoderm']),:]

sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)
sc.pp.highly_variable_genes(adata, min_mean=0.0125, max_mean=3, min_disp=0.5)

adata.raw = adata

adata = adata[:, adata.var.highly_variable]
sc.pp.regress_out(adata, ['total_counts', 'pct_counts_mt'])
sc.pp.scale(adata, max_value=10)
sc.tl.pca(adata, svd_solver='arpack')

sce.pp.harmony_integrate(adata,'sample',max_iter_harmony=20)

sc.pp.neighbors(adata, n_neighbors=50, n_pcs=20, knn=True, use_rep='X_pca_harmony')
sc.tl.leiden(adata,resolution=1.2) 
sc.tl.umap(adata)

In [ ]:
sc.pl.pca_variance_ratio(adata, log=True, n_pcs = 50)

In [ ]:
sc.set_figure_params(dpi=60)

sc.pl.umap(adata, color=['leiden','sample'], title="UMAP Clytia planula",ncols=1)
sc.pl.umap(adata, color=['leiden'], legend_loc= 'on data')
sc.pl.umap(adata, color=['total_counts', 'n_genes_by_counts'])
sc.pl.umap(adata, color=['log1p_total_counts', 'log1p_n_genes_by_counts'])

In [ ]:
# Re-exporting data
adata.write_h5ad(outputPath+'Clytia_planula.h5ad')

In [ ]:
# Ranking and extracting marker genes for all clusters
sc.tl.rank_genes_groups(adata,'leiden',n_genes=1000,method='wilcoxon')

clusters = np.unique(adata.obs['leiden'])
clusters = clusters.astype('int')
clusters = np.sort(clusters)
clusters = clusters.astype('str')

markers = pd.DataFrame()
clus = []
markerGene = []
padj = []
for i in clusters:
  genes = adata.uns['rank_genes_groups']['names'][str(i)]
  clus += list(np.repeat(i,len(genes)))
  markerGene += list(genes)
  padj += list(adata.uns['rank_genes_groups']['pvals_adj'][str(i)])
markers['clus'] = clus
markers['markerGene'] = markerGene
markers['padj'] = padj
significant = []
for index,row in markers.iterrows():
    if row['padj'] < 0.05:
        significant += "T"
    else:
        significant += "F"
markers['significant'] = significant
significant_markers = markers[markers.significant == "T"]
significant_markers.to_csv(outputPath+'Clytia_planula_markers_1000g.csv', sep='\t',index=False)
print(significant_markers)

In [ ]:
def cluster_markers(markersDf,clusterNumber):
    subset = markersDf[markersDf["clus"].isin(clusterNumber)]
    markerGene = subset["markerGene"].tolist()
    return markerGene

def save_markers(markerList,path,clusterName):
    with open(path+"Clytia_adult_"+clusterName+"_markers.txt","w") as f:
        for i in markerList:
            f.write(i+"\n")
        f.close

In [ ]:
# Plotting top marker gene for each cluster
markers = []
for i in np.unique(adata.obs['leiden']):
     markers += [adata.uns['rank_genes_groups']['names'][str(i)][0]]
sc.pl.umap(adata,color=markers,color_map = 'viridis')

In [ ]:
# Synaptotagmin
sc.settings.set_figure_params(dpi=90, facecolor='white')
sc.pl.umap(adata,color=['XLOC_038850','XLOC_006609','XLOC_038825','XLOC_041567','XLOC_007395','XLOC_040916','XLOC_029295','XLOC_029731','XLOC_043438','XLOC_011327','XLOC_011867','XLOC_002849','XLOC_002851','XLOC_000865','XLOC_003662','XLOC_019417'])

In [ ]:
# Cnidocytes
sc.pl.umap(adata,color=['XLOC_017841','XLOC_017845','XLOC_015554','XLOC_044122','XLOC_039385','XLOC_039341','XLOC_004102'],title=['XLOC_017841 znf845-a','XLOC_017845 znf845-b','XLOC_015554 mos3','XLOC_044122 mcol3/4','XLOC_039385 m14-peptidase','XLOC_039341 sans/USH-1G','XLOC_004102 nematocilin'],size=20,ncols=2,color_map = 'viridis')
cnidocyte_markers = cluster_markers(significant_markers,['4','14','28','11','7','16','25']) 
save_markers(cnidocyte_markers,outputPath,"cnidocyte")

In [ ]:
# neurons
sc.pl.umap(adata,color=['XLOC_001566','XLOC_029731','XLOC_004785','XLOC_030971','XLOC_030920','XLOC_018937','XLOC_045293'],title=['XLOC_001566 calmodulin','XLOC_029731 synaptotagmin5','XLOC_004785 ELAV1','XLOC_030971 ELAV2','XLOC_030920 hlh6','XLOC_018937 neurogenin','XLOC_045293 sox10'],size=40,color_map = 'viridis')
neuronal_markers = cluster_markers(significant_markers,['33','31','30','17','13','26']) 
save_markers(neuronal_markers,outputPath,"neuronal")

In [ ]:
# stem/germ
sc.pl.umap(adata,color=['XLOC_004150','XLOC_002094','XLOC_044232','XLOC_043332','XLOC_006164','XLOC_007915','XLOC_039192','XLOC_033976','XLOC_033801'],title=['GFP2a XLOC_004102','Clytin2 XLOC_002094','Oocyte Protein XLOC_044232','Boule1a XLOC_043332','FMNeductase XLOC_006164','Piwi XLOC_007915','XLOC_039192 Nanos2','XLOC_033976 PL10','XLOC_033801 Vasa'],size=20, color_map = 'viridis')

In [ ]:
# Pharyngeal ectoderm
sc.pl.umap(adata,color=['XLOC_044586'],size=20, color_map = 'viridis')

In [ ]:
#Gastroderm markers
sc.pl.umap(adata, color=['XLOC_005609','XLOC_008632','XLOC_006965','XLOC_008858','XLOC_038345','XLOC_010708','XLOC_033751','XLOC_007437','XLOC_029934','XLOC_030379','XLOC_032555'], title=['XLOC_005609 VCBSprot','XLOC_008632 FibCdom-2','XLOC_006965 CathepsinL','XLOC_008858 DDAH','XLOC_038345 CheGast','XLOC_010708 P-lipase','XLOC_033751 Fibulin','XLOC_007437 BP10-like','XLOC_029934 FibColl-A','XLOC_030379 FibColl-B','XLOC_032555 NucHyd'], size=20, color_map='viridis')

In [ ]:
#bioluminescent cells
sc.pl.umap(adata, color=['XLOC_004165'],title=['XLOC_004165 GFP2'],size=20, color_map='viridis')

In [ ]:
#Gland cells
sc.pl.umap(adata,color=['XLOC_006072','XLOC_021506','XLOC_001911','XLOC_034427','XLOC_002272'],title=['XLOC_006072 FibCdom-1','XLOC_021506 C-lectin','XLOC_001911 Tryp-like','XLOC_034427 ShKT-TrypB','XLOC_002272 ShKT-TrypA'], size=20, color_map='viridis')

In [ ]:
# epidermal/muscle from medusa
sc.pl.umap(adata,color=['XLOC_000520','XLOC_042542','XLOC_044475','XLOC_038183','XLOC_029205','XLOC_007598','XLOC_031686'],title=['XLOC_000520 TPM-A','XLOC_042542 TPM-B','XLOC_044475 Peroxidase','XLOC_038183 ST MyHCa','XLOC_029205 ST MyHCb','XLOC_007598 GFP3','XLOC_031686 wnt2'],size=40,color_map='viridis', ncols=2)

In [ ]:
#mixed markers 
sc.pl.umap(adata,color=['XLOC_006965','XLOC_036006','XLOC_007915','XLOC_033801','XLOC_007598','XLOC_001566','XLOC_029731','XLOC_004102'],title=['XLOC_006965 cathepsinL','XLOC_036006 nanos1','XLOC_007915 piwi','XLOC_033801 vasa','XLOC_007598 GFP3','XLOC_001566 calmodulin','XLOC_029731 synaptotagmin5','XLOC_004102 GFP2a'],size=40,color_map = 'viridis')

In [ ]:
# stem cells vasa+
sc.pl.umap(adata, color=['XLOC_033801','XLOC_000173','XLOC_007932'], title=('XLOC_033801 vasa','XLOC_000173','XLOC_007932'), size=40, color_map='viridis')

In [ ]:
# ectoderm v3
sc.pl.umap(adata, color=['XLOC_000520','XLOC_032848','XLOC_000520','XLOC_011667','XLOC_038183','XLOC_035583','XLOC_005114'], title=['XLOC_000520 tropomyosin 1','XLOC_032848','XLOC_000520 TPM-A','XLOC_011667 myosin lc kinase','XLOC_038183 ST MyHCa','XLOC_035583  GFP1','XLOC_005114'], size=40, color_map='viridis')

In [ ]:
# gastroderm v3
sc.pl.umap(adata,color=['XLOC_006965','XLOC_008858','XLOC_038345','XLOC_037915'],title=['XLOC_006965 CathepsinL','XLOC_008858 DDAH','XLOC_038345 cheGAst','XLOC_037915'], size=40, color_map='viridis')

In [ ]:
# neuropeptides
sc.pl.umap(adata,color=['XLOC_017096','XLOC_003691','XLOC_019434','XLOC_012334','XLOC_040580','XLOC_041442','XLOC_041402','XLOC_017097','XLOC_000626','XLOC_004021','XLOC_003339','XLOC_030120','XLOC_040209','XLOC_010892','XLOC_035224','XLOC_021799','XLOC_008730'],title=['XLOC_017096 pp2','XLOC_003691 pp3','XLOC_019434 pp5','XLOC_012334 pp6','XLOC_040580 pp7b','XLOC_041442 pp9a','XLOC_041402 pp9b','XLOC_017097 pp11','XLOC_000626 pp13','XLOC_004021 pp14','XLOC_003339 pp15','XLOC_030120 pp17','XLOC_040209 pp19','XLOC_010892 pp21','XLOC_035224 pp24','XLOC_021799 pp25','XLOC_008730 pp26'],size=40,color_map = 'viridis')

In [ ]:
# Mucous gland
sc.pl.umap(adata,color=['XLOC_043132','XLOC_005440'])

In [ ]:
# Test 
sc.tl.paga(adata, groups='leiden')

In [ ]:
#sc.pl.paga(adata, color=['XLOC_044122','XLOC_039385','XLOC_039341','XLOC_004102'])

In [ ]:
adata.obs['leiden_anno'] = adata.obs['leiden']

In [ ]:
#adata.obs['leiden_anno'].cat.categories = ["0 - Endomesoderm","1 - Secretory","2 - Endomesoderm","3 - Endomesoderm","4 - Endomesoderm","5 - Ectoderm","6 - Ectoderm","7 - NPC","8 - Bioluminescent cells","9 - Secretory","10 - Early cnidocytes","11 - Secretory","12 - Cnidocytes 2","13 - Mature cnidocytes","14 - Cnidocytes 1","15 - Neurons (ecto)","16 - Secretory","17 - Neurons (endomeso)","18 - Neurons (ecto)","19 - Neurons (ecto)","20 - Neurons (endomeso)","21 - Neurons (endomeso)","22 - Neurons (ecto)"]

In [ ]:
sc.tl.paga(adata, groups='leiden_anno')

In [ ]:
sc.settings.set_figure_params(dpi=140, facecolor='white')

In [ ]:
sc.pl.paga(adata, threshold=0.03, show=False,fontsize=5,edge_width_scale=0.5,node_size_scale=0.5)

In [ ]:
sc.settings.set_figure_params(dpi=60, facecolor='white')

In [ ]:
sc.pl.umap(adata, color=['leiden','XLOC_001015'], legend_loc= 'on data')

In [ ]:
# Epidermis
'XLOC_009795','XLOC_031686','XLOC_000520','XLOC_006232','XLOC_045135','XLOC_035583','XLOC_003088'
# i-cells
'XLOC_007915','XLOC_036006','XLOC_033801'
# neurosecretory
'XLOC_043740','XLOC_042849','XLOC_013806'
# Mucous cells
'XLOC_043134','XLOC_014323','XLOC_030920'
# Gastroderm
'XLOC_009611','XLOC_005505','XLOC_007871','XLOC_001844','XLOC_030379','XLOC_040666','XLOC_031568','XLOC_042085','XLOC_006965','XLOC_036949','XLOC_013735','XLOC_001095','XLOC_008858','XLOC_038345'


In [ ]:
# epidermis
sc.pl.umap(adata, color=['XLOC_009795','XLOC_031686','XLOC_000520','XLOC_006232','XLOC_045135','XLOC_035583','XLOC_003088'], legend_loc= 'on data')

In [ ]:
# i-cells
sc.pl.umap(adata, color=['XLOC_007915','XLOC_036006','XLOC_033801'])

In [ ]:
# Neurosecretory
sc.pl.umap(adata, color=['XLOC_043740','XLOC_042849','XLOC_013806'])

In [ ]:
# Mucous gland
sc.pl.umap(adata, color=['XLOC_043134','XLOC_014323','XLOC_030920'])

In [ ]:
# Gastroderm 
sc.pl.umap(adata, color=['XLOC_009611','XLOC_005505','XLOC_007871','XLOC_001844','XLOC_030379','XLOC_040666','XLOC_031568','XLOC_042085','XLOC_006965','XLOC_036949','XLOC_013735','XLOC_001095','XLOC_008858','XLOC_038345'])

In [ ]:
# PEC
sc.pl.umap(adata, color=['XLOC_038633','XLOC_037406','XLOC_037910','XLOC_009140','XLOC_013080','XLOC_021374'])

In [ ]:
# Neural
sc.pl.umap(adata, color=['XLOC_030971','XLOC_041442','XLOC_004021','XLOC_019434','XLOC_034880'])

In [ ]:
cluster_markers(significant_markers,['5'])

In [ ]:
adata.obs['leiden_anno'].cat.categories = ["0 - I-cells","1 - Epidermis","2 - Epidermis","3 - Epidermis","4 - Gastrodermis","5 - Epidermis","6 - I-cells","7 - Early cnidocytes","8 - Mucous gland cells","9 - Neurons","10 - Cnidocytes 2","11 - I-cells","12 - Mature cnidocytes","13 - Cnidocytes 1","14 - Gastrodermis","15 - Neurosecretory","16 - Neurons","17 - PEC-B","18 - PEC-A"]

In [ ]:
sc.settings.set_figure_params(dpi=90, facecolor='white')
sc.pl.umap(adata, color=['leiden_anno'],title='Clytia planula annotated clusters',ncols=1,frameon=False)

In [ ]:
# Exporting for SCVI

adata.write_h5ad(outputPath+'Clytia_planula_annotated.h5ad')

In [ ]:
sc.settings.set_figure_params(dpi=150, facecolor='white')
sc.pl.umap(adata,title='Clytia planula\nRBH - one2one - orthopairsBased', color=['leiden'], legend_loc= 'on data', legend_fontsize=6, legend_fontoutline=3, palette={"0":"#E2E2E2", "1":"#E2E2E2", "2":"#E2E2E2", "3":"#E2E2E2", "4":"#E2E2E2", "5":"#E2E2E2", "6":"#E2E2E2", "7":"#A366E0", "8":"#E2E2E2", "9":"#E2E2E2", "10":"#A366E0", "11":"#E2E2E2", "12":"#A366E0", "13":"#A366E0", "14":"#E2E2E2", "15":"#E2E2E2", "16":"#E2E2E2", "17":"#E2E2E2", "18":"#E2E2E2"}, frameon=False)

In [ ]:
sc.pl.umap(adata,title='Clytia planula\nRBH - many2many - orthopairsBased', color=['leiden'], legend_loc= 'on data', legend_fontsize=6, legend_fontoutline=3, palette={"0":"#E2E2E2", "1":"#E2E2E2", "2":"#E2E2E2", "3":"#E2E2E2", "4":"#E2E2E2", "5":"#E2E2E2", "6":"#E2E2E2", "7":"#A366E0", "8":"#E2E2E2", "9":"#E2E2E2", "10":"#A366E0", "11":"#E2E2E2", "12":"#A366E0", "13":"#A366E0", "14":"#E2E2E2", "15":"#E2E2E2", "16":"#E2E2E2", "17":"#E2E2E2", "18":"#E2E2E2"}, frameon=False)

In [ ]:
sc.pl.umap(adata,title='Clytia planula\nOrthoFinder - many2many - orthopairsBased', color=['leiden'], legend_loc= 'on data', legend_fontsize=6, legend_fontoutline=3, palette={"0":"#E2E2E2", "1":"#E2E2E2", "2":"#E2E2E2", "3":"#E2E2E2", "4":"#E2E2E2", "5":"#E2E2E2", "6":"#E2E2E2", "7":"#A366E0", "8":"#E2E2E2", "9":"#E2E2E2", "10":"#A366E0", "11":"#E2E2E2", "12":"#A366E0", "13":"#A366E0", "14":"#E2E2E2", "15":"#E2E2E2", "16":"#E2E2E2", "17":"#E2E2E2", "18":"#E2E2E2"}, frameon=False)

In [ ]:
sc.pl.umap(adata,title='Clytia planula\nOrthoFinder - many2many - genomeBased', color=['leiden'], legend_loc= 'on data', legend_fontsize=6, legend_fontoutline=3, palette={"0":"#E2E2E2", "1":"#E2E2E2", "2":"#E2E2E2", "3":"#E2E2E2", "4":"#E2E2E2", "5":"#E2E2E2", "6":"#E2E2E2", "7":"#A366E0", "8":"#E2E2E2", "9":"#E2E2E2", "10":"#A366E0", "11":"#E2E2E2", "12":"#A366E0", "13":"#A366E0", "14":"#E2E2E2", "15":"#E2E2E2", "16":"#E2E2E2", "17":"#E2E2E2", "18":"#E2E2E2"}, frameon=False)

In [ ]:
sc.pl.umap(adata,title='Clytia planula\nRBH - one2one - orthopairsBased', color=['leiden'], legend_loc= 'on data', legend_fontsize=6, legend_fontoutline=3, palette={"0":"#E2E2E2", "1":"#E2E2E2", "2":"#E2E2E2", "3":"#E2E2E2", "4":"#E2E2E2", "5":"#E2E2E2", "6":"#E2E2E2", "7":"#E2E2E2", "8":"#E2E2E2", "9":"#0080ff", "10":"#E2E2E2", "11":"#E2E2E2", "12":"#E2E2E2", "13":"#E2E2E2", "14":"#E2E2E2", "15":"#0080ff", "16":"#0080ff", "17":"#E2E2E2", "18":"#E2E2E2"}, frameon=False)

In [ ]:
sc.pl.umap(adata,title='Clytia planula\nRBH - many2many - orthopairsBased', color=['leiden'], legend_loc= 'on data', legend_fontsize=6, legend_fontoutline=3, palette={"0":"#E2E2E2", "1":"#E2E2E2", "2":"#E2E2E2", "3":"#E2E2E2", "4":"#E2E2E2", "5":"#E2E2E2", "6":"#E2E2E2", "7":"#E2E2E2", "8":"#E2E2E2", "9":"#0080ff", "10":"#E2E2E2", "11":"#E2E2E2", "12":"#E2E2E2", "13":"#E2E2E2", "14":"#E2E2E2", "15":"#0080ff", "16":"#0080ff", "17":"#E2E2E2", "18":"#E2E2E2"}, frameon=False)

In [ ]:
sc.pl.umap(adata,title='Clytia planula\nOrthoFinder - many2many - orthopairsBased', color=['leiden'], legend_loc= 'on data', legend_fontsize=6, legend_fontoutline=3, palette={"0":"#E2E2E2", "1":"#E2E2E2", "2":"#E2E2E2", "3":"#E2E2E2", "4":"#E2E2E2", "5":"#E2E2E2", "6":"#E2E2E2", "7":"#E2E2E2", "8":"#E2E2E2", "9":"#0080ff", "10":"#E2E2E2", "11":"#E2E2E2", "12":"#E2E2E2", "13":"#E2E2E2", "14":"#E2E2E2", "15":"#0080ff", "16":"#0080ff", "17":"#E2E2E2", "18":"#E2E2E2"}, frameon=False)

In [ ]:
sc.pl.umap(adata,title='Clytia planula\nOrthoFinder - many2many - genomeBased', color=['leiden'], legend_loc= 'on data', legend_fontsize=6, legend_fontoutline=3, palette={"0":"#E2E2E2", "1":"#E2E2E2", "2":"#E2E2E2", "3":"#E2E2E2", "4":"#E2E2E2", "5":"#E2E2E2", "6":"#E2E2E2", "7":"#E2E2E2", "8":"#E2E2E2", "9":"#0080ff", "10":"#E2E2E2", "11":"#E2E2E2", "12":"#E2E2E2", "13":"#E2E2E2", "14":"#E2E2E2", "15":"#0080ff", "16":"#0080ff", "17":"#E2E2E2", "18":"#E2E2E2"}, frameon=False)